In [452]:
from flat import document, rgb, rgba, font, strike, shape
from flat.command import moveto, quadto, curveto, lineto, closepath
from bezmerizing import Polyline, Path
import numpy as np
from copy import copy

In [453]:
from numpy.random import uniform, normal, choice

In [454]:
from IPython.display import SVG, display
def show(page):
    display(SVG(page.svg()))

In [455]:
f = font.open("./NotoSans-Regular.ttf")

In [456]:
def glyphcommands(f, ch):
    return Path([copy(cmd) for cmd in f.glyph(f.charmap[ord(ch)])])
def advancefor(f, ch):
    return f.advances[f.charmap[ord(ch)]]

In [457]:
def nudge(cmd, x, y):
    c = copy(cmd) 
    if type(cmd) in (moveto, lineto, quadto, curveto):
        c.x += x
        c.y += y
    if type(cmd) in (quadto, curveto):
        c.x1 += x
        c.y1 += y
    if type(cmd) == curveto:
        c.x2 += x
        c.y2 += y
    return c

In [458]:
w = "melancholy"

In [459]:
page = document(300,500, 'pt').addpage()
pen = shape().stroke(rgb(40, 40, 40)).fill(rgb(0,0,0))
size = 36 / f.density
cx = 0
cy = 36
n = 10

for line in range(n):
    for ch in w:
        glyph_path = (glyphcommands(f,ch)
                      .scale(size)
                      .translate(cx, cy + normal(0,4)))
        glyph_path_cmds = []
        for cmd in glyph_path:
            if hasattr(cmd, 'x'):
                glyph_path_cmds.append(nudge(cmd,x=cmd.x*0.5, y = 0))
            else:
                glyph_path_cmds.append(cmd)
        glyph_path_mod = Path(glyph_path_cmds)
        page.place(pen.path(glyph_path_mod.translate(10,40).scale(1)))
        cx += advancefor(f, ch) * size
    cx = 0
    cy += 50
show(page)


In [505]:
w = "motion"

In [521]:
page = document(300,500, 'pt').addpage()
pen = shape().stroke(rgb(40, 40, 40)).fill(rgb(0,0,0))
size = 36 / f.density
cx = 0
cy = 36
n = 1

for line in range(n):
    for ch in w:
        glyph_path = (glyphcommands(f,ch)
                      .scale(1)
                      .translate(cx, cy))
        cx += advancefor(f, ch) 
        glyph_path_cmds = []
        y_min = f.ascender * -0.3
        y_max = f.ascender * -0.1
        for cmd in glyph_path:
            if hasattr(cmd, 'y') and cmd.y > y_min and cmd.y < y_max:
                glyph_path_cmds.append(nudge(cmd,x = 200, y = 0))
            else:
                glyph_path_cmds.append(cmd)
        glyph_path_mod = Path(glyph_path_cmds)
        page.place(pen.path(glyph_path_mod.translate(0,1000 + normal(0,100)).scale(0.1)))
    cx = 0
    cy += 800
show(page)

In [496]:
from itertools import chain
def combine_path(f, s):
    text_paths = []
    cx = 0
    for ch in s:
        glyph_path = glyphcommands(f, ch).translate(cx, 0)
        text_paths.append(glyph_path)
        cx += advancefor(f, ch)
    combined = Path(list(chain(*text_paths)))
    return combined

In [518]:
qf_path_orig = combine_path(f, "motion")
qf_path_cmds = []
for cmd in qf_path_orig:
    y_min = f.ascender * -0.3
    y_max = f.ascender * -0.1
    if hasattr(cmd, 'y') and cmd.y > y_min and cmd.y < y_max:
        qf_path_cmds.append(nudge(cmd,
                                  x=200,
                                  y=0))
    else:
        qf_path_cmds.append(cmd)
qf_path_mod = Path(qf_path_cmds)

In [519]:
page = document(700,500, 'pt').addpage()
pen = shape().nostroke().fill(rgb(0,0,0))
size = 72 / f.density
cy = 72
n = 4
i = 1

for line in range(n):
    page.place(pen.path(qf_path_mod.scale(factor*i).translate(0, cy)))
    cy +=72*i
    i *=1.2
show(page)